In [ ]:
import speech_recognition as sr
import pyttsx3
import requests
import json
import re
import os
import subprocess
import webbrowser
import pygetwindow as gw
import time
import pywhatkit

# === Configuration ===
API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://your-site-or-project.com",
    "X-Title": "Dada Assistant"
}

# === Initialize TTS Engine ===
engine = pyttsx3.init()
engine.setProperty("rate", 180)
engine.setProperty("volume", 1.0)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

# === Initialize Recognizer ===
recognizer = sr.Recognizer()
microphone = sr.Microphone()

def callback(recognizer, audio):
    try:
        command = recognizer.recognize_google(audio, language='en-IN')
        if command.lower() == "stop":
            engine.stop()
            print("Speech stopped by user")
    except:
        pass

stop_listening = recognizer.listen_in_background(microphone, callback)

# === Speak Function ===
def speak(text, allow_interruption=False):
    if allow_interruption:
        stop_listening(wait_for_stop=False)
        recognizer.listen_in_background(microphone, callback)
    engine.say(text)
    engine.runAndWait()

# === Listen Function ===
def listen():
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source, duration=0.3)
        print("🎤 Listening...")
        try:
            audio = recognizer.listen(source, timeout=3, phrase_time_limit=5)
            print("🔍 Recognizing...")
            query = recognizer.recognize_google(audio, language='en-IN')
            print("You said:", query)
            return query
        except sr.WaitTimeoutError:
            print("⏱️ Timeout: You didn’t speak in time.")
            return None
        except sr.UnknownValueError:
            print("❌ Could not understand audio.")
            return None
        except sr.RequestError as e:
            print(f"❌ Could not request results; {e}")
            return None

# === Clean AI Response ===
def clean_response(text):
    text = re.sub(r'\\n|\n|\r', ' ', text)
    text = re.sub(r'[*_#>\[\]{}|]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()

# === Talk to OpenRouter AI ===
def chat_with_deepseek(prompt):
    try:
        data = {
            "model": "deepseek-mixtral-8x7b-instruct",  # ✅ FIXED model name
            "messages": [
                {"role": "system", "content": "You are Dada, an intelligent AI assistant."},
                {"role": "user", "content": prompt}
            ]
        }
        response = requests.post(API_URL, headers=HEADERS, data=json.dumps(data))
        if response.status_code == 200:
            result = response.json()
            raw_answer = result["choices"][0]["message"]["content"]
            print("🧠 AI Raw Response:", raw_answer)
            return clean_response(raw_answer)
        else:
            print("❌ API Error:", response.status_code, response.text)
            return "invalid"
    except Exception as e:
        print("❌ Exception:", str(e))
        return "invalid"

# === System and App Commands ===
def shutdown():
    speak("Shutting down the system.")
    os.system("shutdown /s /t 1")

def open_chrome():
    speak("Opening Chrome.")
    path = r"C:\\ProgramData\\Microsoft\\Windows\\Start Menu\\Programs\\Google Chrome.lnk"
    if os.path.exists(path):
        os.startfile(path)
    else:
        speak("Sorry, I couldn't find the Chrome shortcut.")

def open_whatsapp():
    speak("Opening WhatsApp.")
    path = r"C:\\Users\\svart\\OneDrive\\Documents\\Desktop\\WhatsApp - Shortcut.lnk"
    if os.path.exists(path):
        os.startfile(path)
        time.sleep(5)
        try:
            for window in gw.getWindowsWithTitle("WhatsApp"):
                if window.isMinimized:
                    window.restore()
                window.activate()
                window.maximize()
                break
            else:
                speak("WhatsApp window not found.")
        except Exception as e:
            print("⚠️ Exception while bringing WhatsApp to front:", e)
            speak("WhatsApp is open, but I couldn't bring it to the front.")
    else:
        speak("Sorry, I can't find the WhatsApp shortcut.")

def search_google(query):
    speak(f"Searching Google for {query}")
    url = f"https://www.google.com/search?q={query}"
    webbrowser.open(url)

def open_folder(folder_name):
    speak(f"Opening folder {folder_name}")
    folder_path = os.path.join("C:\\Users\\svart\\", folder_name)
    if os.path.exists(folder_path):
        os.startfile(folder_path)
    else:
        speak("Sorry, I can't find that folder.")

# === Play YouTube Video ===
def play_youtube(query):
    speak(f"Playing {query} on YouTube.")
    try:
        pywhatkit.playonyt(query)
    except Exception as e:
        print("❌ Error playing YouTube video:", e)
        speak("Sorry, I couldn't play the video.")

# === Main Loop ===
if __name__ == "__main__":
    speak("Hello, I am Dada. Say 'Dada' to activate me.", allow_interruption=False)

    while True:
        print("🕒 Waiting for wake word 'Dada'...")
        wake_input = listen()

        if wake_input and "dada" in wake_input.lower():
            speak("Yes Dear? What would you like me to do?", allow_interruption=False)
            command = listen()

            if command:
                command_lower = command.lower()

                if any(word in command_lower for word in ["exit", "quit", "stop", "bye"]):
                    speak("Goodbye! Have a great day.", allow_interruption=False)
                    break

                elif "shutdown" in command_lower:
                    shutdown()
                    break

                elif "open chrome" in command_lower:
                    open_chrome()

                elif "open whatsapp" in command_lower:
                    open_whatsapp()

                elif "search for" in command_lower or "google" in command_lower:
                    search_query = command_lower.replace("search for", "").replace("google", "").strip()
                    if search_query:
                        search_google(search_query)
                    else:
                        speak("What should I search for?")

                elif "open folder" in command_lower:
                    folder = command_lower.replace("open folder", "").strip()
                    open_folder(folder)

                elif "youtube" in command_lower:  # ✅ Improved YouTube trigger
                    yt_query = command_lower.replace("play", "").replace("open", "").replace("on youtube", "").replace("from youtube", "").replace("youtube", "").strip()
                    if yt_query:
                        play_youtube(yt_query)
                    else:
                        speak("Which video do you want to play?")

                elif "cancel" in command_lower or "stop talking" in command_lower:
                    speak("Okay, stopping the assistant.")
                    break

                else:
                    response = chat_with_deepseek(command)
                    if response and len(response.strip()) > 3 and "invalid" not in response.lower():
                        speak(response, allow_interruption=True)
                    else:
                        speak("Sorry, that response was unclear or invalid. Stopping for now.")
            else:
                speak("Sorry, I didn't catch that.", allow_interruption=False)


🕒 Waiting for wake word 'Dada'...
🎤 Listening...
🔍 Recognizing...
You said: Dada
🎤 Listening...
🔍 Recognizing...
You said: open Kathal song on YouTube
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
🔍 Recognizing...
❌ Could not understand audio.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
🔍 Recognizing...
❌ Could not understand audio.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
🔍 Recognizing...
❌ Could not understand audio.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
🔍 Recognizing...
❌ Could not understand audio.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
⏱️ Timeout: You didn’t speak in time.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
⏱️ Timeout: You didn’t speak in time.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
⏱️ Timeout: You didn’t speak in time.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
⏱️ Timeout: You didn’t speak in time.
🕒 Waiting for wake word 'Dada'...
🎤 Listening...
⏱️ Timeout: You didn’t speak in time.
🕒 Waiting for wake word 'Dada'